# Load Ni-Mo data

In [1]:
from pymatgen import Structure
from monty.serialization import loadfn

data = loadfn('data.json')
train_structures = [d['structure'] for d in data]
train_energies = [d['outputs']['energy'] for d in data]
train_forces = [d['outputs']['forces'] for d in data]

# Set up the GAP and train

In [2]:
from maml.apps.pes import GAPotential

gap = GAPotential()
gap.train(train_structures=train_structures, train_energies=train_energies,
          train_forces=train_forces, sparse_method='cur_points', 
          l_max=8, n_max=8, e0=-0.97, n_sparse=200, cutoff=5.0)

0

# Predict the energies, forces of training data

In [3]:
df_orig, df_predict = gap.evaluate(test_structures=train_structures, 
                                   test_energies=train_energies,
                                   test_forces=train_forces, test_stresses=None)

In [4]:
df_predict[df_predict['dtype'] == 'energy']

,y_orig,n,dtype
0,-1022.578431,144.0,energy
433,-1021.852543,144.0,energy
866,-967.381901,144.0,energy
1299,-973.891519,144.0,energy
1732,-968.058261,144.0,energy
2165,-1009.151963,144.0,energy
2598,-1021.188015,144.0,energy
3031,-959.079041,144.0,energy
3464,-1008.984483,144.0,energy
3897,-972.192187,144.0,energy


# Lattice constant, Elastic constant
### Large error due to limited training data -- 10 structures

In [ ]:
from pymatgen.core import Lattice

Ni = Structure.from_spacegroup(sg='Fm-3m', species=['Ni'], lattice=Lattice.cubic(3.51), coords=[[0, 0, 0]])
Mo = Structure.from_spacegroup(sg='Im-3m', species=['Mo'], lattice=Lattice.cubic(3.17), coords=[[0, 0, 0]])

In [8]:
from maml.apps.pes import LatticeConstant

Ni = Structure.from_spacegroup(sg='Fm-3m', species=['Ni'], lattice=Lattice.cubic(3.51), coords=[[0, 0, 0]])
lc_calculator = LatticeConstant(ff_settings=gap)
a, b, c = lc_calculator.calculate([Ni])[0]
print('Ni', 'Lattice a: {}, Lattice b: {}, Lattice c: {}'.format(a, b, c))

Ni Lattice a: 3.36261678052108, Lattice b: 3.36261678052108, Lattice c: 3.36261678052108


In [9]:
Mo = Structure.from_spacegroup(sg='Im-3m', species=['Mo'], lattice=Lattice.cubic(3.17), coords=[[0, 0, 0]])
lc_calculator = LatticeConstant(ff_settings=gap)
a, b, c = lc_calculator.calculate([Mo])[0]
print('Mo', 'Lattice a: {}, Lattice b: {}, Lattice c: {}'.format(a, b, c))

Mo Lattice a: 3.13154097650104, Lattice b: 3.13154097650104, Lattice c: 3.13154097650104


In [5]:
from maml.apps.pes import ElasticConstant

Ni_ec_calculator = ElasticConstant(ff_settings=gap)
Ni_C11, Ni_C12, Ni_C44, _ = Ni_ec_calculator.calculate([Ni])[0]
print('Ni', ' C11: ', Ni_C11, 'C12: ', Ni_C12, 'C44: ', Ni_C44)

Ni  C11:  294.00292058499 C12:  215.956808932469 C44:  172.632763694098


In [6]:
Mo_ec_calculator = ElasticConstant(ff_settings=gap)
Mo_C11, Mo_C12, Mo_C44, _ = Mo_ec_calculator.calculate([Mo])[0]
print('Mo', ' C11: ', Mo_C11, 'C12: ', Mo_C12, 'C44: ', Mo_C44)

Mo  C11:  1178.51552223199 C12:  -428.408236874733 C44:  373.512164542


# Load from parameters files

In [11]:
gap_loaded = GAPotential.from_config(filename='gap.xml')

# Energy, force, stress prediction

In [13]:
from maml.apps.pes import EnergyForceStress

efs_calculator = EnergyForceStress(ff_settings=gap_loaded)
energy, forces, stresses = efs_calculator.calculate([train_structures[0]])[0]

print('energy: {}'.format(energy))
print('forces: \n', forces)
print('stresses: ', stresses)

energy: -1022.57843361339
forces: 
 [[-0.148907    0.691195    0.00622235]
 [ 0.553757   -0.299642   -0.674699  ]
 [-0.530246   -0.11432     0.578683  ]
 [ 0.229872    0.518665   -0.43865   ]
 [-0.599489   -0.0095873  -1.50022   ]
 [ 0.521754    0.16086    -0.357313  ]
 [-0.74472     0.22597     0.243968  ]
 [-0.0397528   0.0397773   0.800801  ]
 [-1.32847    -0.50867     0.247439  ]
 [ 0.745455   -0.32021    -0.542783  ]
 [ 0.110386   -0.085871   -0.0303303 ]
 [-0.79898     0.151923   -0.567254  ]
 [-0.124843   -0.129319    0.542627  ]
 [-0.200481   -0.599946    0.0196301 ]
 [ 0.336649   -0.725849   -1.21457   ]
 [-0.558773   -0.494143    0.111651  ]
 [-0.130768   -0.0386151  -0.195661  ]
 [-0.26047     0.023753   -0.613401  ]
 [-0.0526436  -0.509165    1.20497   ]
 [-0.455376    0.166561   -0.403331  ]
 [-0.779056    0.547719    0.437108  ]
 [-0.254392   -0.0719912   0.226387  ]
 [-0.298298    1.59229     0.705392  ]
 [ 1.09219    -0.392169   -0.275389  ]
 [-0.0908315  -0.647851    0